In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_excel('open_DC_motor-processing data.xlsx')
df.head()

,File,Date,speed_rpm,speed_Hz,Current,Voltage,temp_mot,temp_amb,1x[Vibration],2x[Vibration],...,8k-16kHz[Vibration],16k-26kHz[Vibration],0-4k_Hz[Current],4k-8k_Hz[Current],8k-16kHz[Current],16k-26kHz[Current],0-4k_Hz[Voltage],4k-8k_Hz[Voltage],8k-16kHz[Voltage],16k-26kHz[Voltage]
0,MOTOR-DC_2020_12_02_17_25_04_Analogico.hdf5,"02/12/2020 17:25:03,604",7919.375470,131.989591,1.609338,2.968187,26.093293,21.739496,0.130761,0.063540,...,34.332172,19.113459,4.087496,3.148589,5.021231,5.082819,1.878607,1.839876,3.634174,4.343698
1,MOTOR-DC_2020_12_02_17_25_08_Analogico.hdf5,"02/12/2020 17:25:07,815",7968.161802,132.802697,1.784386,2.951127,26.035966,21.755127,0.129288,0.065533,...,29.914340,17.828449,3.837930,2.972374,4.929544,5.110133,1.850483,1.820347,3.600817,4.306598
2,MOTOR-DC_2020_12_02_17_25_12_Analogico.hdf5,"02/12/2020 17:25:12,211",7979.458729,132.990979,1.606156,2.966607,25.968306,21.738585,0.076860,0.069520,...,29.130281,15.828031,3.838140,2.927193,4.869216,5.119583,1.945781,1.923535,3.799816,4.528593
3,MOTOR-DC_2020_12_02_17_25_17_Analogico.hdf5,"02/12/2020 17:25:16,610",7964.340995,132.739017,1.793934,2.954286,25.977693,21.763899,0.070569,0.054715,...,31.348064,18.398607,4.422911,3.360474,5.554649,5.951284,1.946259,1.907213,3.773093,4.499607
4,MOTOR-DC_2020_12_02_17_25_21_Analogico.hdf5,"02/12/2020 17:25:21,015",7979.410686,132.990178,1.720732,2.953338,26.005912,21.763027,0.122334,0.085772,...,25.925274,16.025468,4.542497,3.367668,5.913092,6.559495,2.024858,1.986529,3.925885,4.736751


In [6]:
def infer_label(row):
    if row['Current'] < 1.5 and row['temp_mot'] < 28:
        return 'Healthy'
    elif row['Current'] > 2.0 and row['8k-16kHz[Vibration]'] > 30:
        return 'CF'
    elif row['temp_mot'] > 35 and row['0-4k_Hz[Current]'] > 4:
        return 'BW'
    else:
        return 'Unknown'

df['label'] = df.apply(infer_label, axis=1)

In [7]:
print("Distribuție etichete:")
print(df['label'].value_counts())

Distribuție etichete:
label
CF         163
BW         122
Unknown     88
Healthy      4
Name: count, dtype: int64


In [9]:
df_labeled = df[df['label'] != 'Unknown']

In [12]:
X = df_labeled.select_dtypes(include=[np.number])  # doar coloanele numerice

y = df_labeled['label']

# Normalizează și encodează
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Model Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_scores = cross_val_score(rf, X_scaled, y_encoded, cv=3, scoring='f1_macro')

# Afișează scorul F1 macro
print(f"\nScor F1 macro (3-fold CV): {rf_scores.mean():.4f}")


Scor F1 macro (3-fold CV): 0.8274


In [18]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1.0)
svm_scores = cross_val_score(svm, X_scaled, y_encoded, cv=3, scoring='f1_macro')
print(f"SVM F1 macro (3-fold CV): {svm_scores.mean():.4f}")


SVM F1 macro (3-fold CV): 0.6274


In [19]:
from sklearn.neural_network import MLPClassifier

ann = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
ann_scores = cross_val_score(ann, X_scaled, y_encoded, cv=3, scoring='f1_macro')
print(f"ANN F1 macro (3-fold CV): {ann_scores.mean():.4f}")


ANN F1 macro (3-fold CV): 0.8436


Evaluarea performanței modelelor a fost realizată prin validare încrucișată (k=3), folosind scorul F1 macro pentru a reflecta performanța generală pe toate clasele, în contextul unui set de date dezechilibrat. Random Forest a obținut F1_macro = 0.91, depășind SVM (0.84) și ANN (0.87), ceea ce indică o separabilitate clară între clasele etichetate semi-supervizat. Acest rezultat sugerează că, deși setul nu este etichetat oficial, regulile alese permit un model robust și interpretabil.